# Lecture 4: Execution on noisy quantum hardware
from IBM Quantum Computing QGSS2024

이번 강의에서는 Qiskit Pattern의 4단계 중에서, 실제로 Qiskit Runtime Primitives를 사용하여 Quantum hardware에서 실행하는 과정에 대해 배우고자한다. 그러나 실제 구현된 Quantum Computer는 이상적인 가정과 다르게 다양한 종류의 noise를 포함하고 있기 때문에 이 noise가 발생하지 않도록 하거나, noise가 발생해도 이를 없앨 수 있는 방법들을 활용해야한다.  

크게 다음과 같은 3가지 방식을 사용하여 quantum system에서 발생하는 noise를 해결한다. 이번 강의에서는 그중에서도 **Suppression, Mitigation**에 대해 다룬다.

- Suppression
    - error를 줄이거나/피하기위한 방법
    - 실행하기 전이나 실행중에 적용한다.
- Mitigation
    - 이미 발생한 error의 영향을 줄이거나/없애기위한 방법
    - 실행이 끝난 뒤나 실행중에 적용한다.
- Correction
    - error가 발생했을 때, 이를 즉시 탐지하고 바로 error를 없애기 위한 방법
    - 실행 중에 적용한다.

원래 기본적으로 Qiskit Runtime Primitive에는 error제거를 위한 방법이 제공되지만, 각 방식을 직접 적용하기 위해 다음과 같이 `optimization_level`, `resilience_level`을 모두 0으로 설정한 primitives를 이용한다.

In [11]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy()

sampler = Sampler(backend, options=None)
estimator = Estimator(backend, options=None)

#! options 추가 방법 1
# Deactivate circuit optimization & error mitigation
estimator.options.optimization_level = 0
estimator.options.resilience_level = 0

## Error Suppression

Error Suppression에는 다음 2가지 종류의 technique이 존재한다.

##### Dynamical Decoupling (DD)
- 회로의 일부분에서 원하지 coupling(=서로 다른 system의 상호작용)이 발생하는 현상인 **cross talk**에 의한 현상을 제거하기 위한 방법
- operator가 적용된 후, IDLE 상태인 qubit와 physically 근처에 있는 qubit에서 operation이 수행되면 cross talk가 발생할 수 있다.
- 따라서 IDLE 상태인 qubit에 일부러 gate를 추가하여 IDLE하지 않게 만든다. 이때, 추가한 gate $U$에 대해 그 연산을 취소 할 수 있는 gate $U^{-1}$역시 함께 적용하여 DD 전후 연산 결과에 변함이 없게 만들어준다.
- 새로운 gate를 삽입하는 것이기 때문에 **gate error**를 추가적으로 야기시킬 수 있으므로 주의하여 사용해야한다.


##### Randomized compiling (twirling)
- gate 연산으로 인해 발생하는 **gate-error**를 제거하기 위한 방법
- **arbitrary noise channel**을 Pauli basis에 projection 시키기 위해, 기존 gate의 양옆에 추가적인 Pauli operator를 삽입한다. (이 Pauli operator의 종류는 랜덤하게 선택된다.)
- Randomized compiling을 수행하면, operation 결과는 변하지 않고 noise만 pauli basis에 projection 시키게 된다. noise channel을 pauli basis에 projection하면 추후 **noise를 제거할 때; Error Mitigation**에서 이를 활용할 수 있다.

### Dynamical Decoupling via qiskit ibm runtime

In [12]:
from qiskit_ibm_runtime import SamplerOptions, EstimatorOptions

#! options 추가 방법 2
options = SamplerOptions()
options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure Dynamical Decoupling
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = 'XX'  # 회로에 삽입할 gate. XX말고 다른 gate들도 사용할 수 있다.
options.dynamical_decoupling.extra_slack_distribution = 'middle'
options.dynamical_decoupling.scheduling_method = 'alap'

### Randomized compiling (twirling); Pauli twirling(PT) via qiskit imb runtime

In [13]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure Twirling
options.twirling.enable_gates = True
options.twirling.enable_measure = False  # measurement gate에도 twirling을 적용할 수 있다. (ZNE를 위해)
options.twirling.num_randomizations = 'auto'
options.twirling.shots_per_randomization = 'auto'
options.twirling.strategy = 'active-accum'

## Error mitigation

Error mitigation에는 다음 2가지 종류의 technique이 존재한다.

##### Twirled readout error extinction (TREX)
- 회로 마지막에서 qubit의 값을 측정할 때, 발생하는 readout error (SPAM error의 일종)를 해결하기 위한 방법
- 실제로 준비된 state(row)와 측정된 state(column)간의 확률을 나타내는 **error matrix**를 만들 수 있다. 이 error matrix $E$에 대한 inverse matrix $E^{-1}$를 곱해주면 그 값이 소거되어 readout error를 제거할 수 있다.
- 그러나 qubit의 개수가 커질수록 error matrix의 크기또한 exponential 하게 증가하기 때문에 inverse matrix를 쉽게 계산할 방법이 필요하다.
- 아이디어: *Error suppression*단계에서 사용하는 **twirling**을 이용하면, noise channel의 matrix를 **Digonal**하게 만들 수 있기 때문에 역행렬을 쉽게 계산할 수 있다.


##### Zero noise extrapolation (ZNE) 🔥
- gate 연산으로 인해 발생하는 **gate-error**를 제거하기 위한 방법
- 주어진 quantum computer가 갖는 noise-level을 1이라고 생각하고, 더 심한 noise를 강제로 만들어내서 noise-level을 증가시켜감에 따라 측정결과가 어떻게 달라지는지를 그래프로 그려 분석한다. (noise amplification phase)
- 그 후, noise-level에 따른 측정값에 대한 함수를 추측하여 **zero-noise level**에서 observable의 expectation value를 추정한다. (extrapolation phase)
- 더 심한 noise를 만들기 위해 다양한 방법이 연구되고 있다. (on-going research)
    - Pulse stretching *(Kandala et al. Nature (2019))*
    - Gate folding *(Shultz et al. PRA (2022))*
    - Probabilistic error amplification *(Li & Benjamin. PRX (2017))*

### Twirled readout error extinction (TREX) via qiskit ibm runtime

In [14]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure TREX
options.resilience.measure_mitigation = True
options.resilience.measure_noise_learning.num_randomizations =  32
options.resilience.measure_noise_learning.shots_per_randomization = 'auto'

# Automatically set by TREX
options.twirling.enable_measure = True

### Zero noise extrapolation (ZNE) via qiskit ibm runtime

In [15]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure ZNE
options.resilience.zne_mitigation = True
options.resilience.zne.noise_factors = (1, 3, 5)                    # noise level
options.resilience.zne.extrapolator = ('exponential', 'linear')     # interpolate method

### Probabilistic error amplification (PEA) via qiskit ibm runtime
*Noise amplification technique for ZNE*

In [16]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure ZNE with PEA
options.resilience.zne_mitigation = True
options.resilience.zne.noise_factors = (1, 3, 5)                    # noise level
options.resilience.zne.extrapolator = 'exponential'                 # interpolate method

# Configure PEA
options.experimental = {
    'resilience': {
        'zne': {
            'amplifier': 'pea'
        }
    }
}

options.resilience.layer_noise_learning.max_layers_to_learn = 4
options.resilience.layer_noise_learning.num_randomizations = 32
options.resilience.layer_noise_learning.shots_per_randomization = 128
options.resilience.layer_noise_learning.layer_pair_depths = (0, 1, 2, 4, 16, 32)

## Combining techniques *via Qiskit IBM Runtime*

In [17]:
from qiskit_ibm_runtime import EstimatorOptions

options = EstimatorOptions(optimization_level=0, resilience_level=0)

# Configure Dynamical Decoupling
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = 'XX'  
options.dynamical_decoupling.extra_slack_distribution = 'middle'
options.dynamical_decoupling.scheduling_method = 'alap'

# Configure Twirling
options.twirling.enable_gates = True
options.twirling.enable_measure = True         # Needed for TREX
options.twirling.num_randomizations = 'auto'
options.twirling.shots_per_randomization = 'auto'
options.twirling.strategy = 'active-accum'

# Configure TREX
options.resilience.measure_mitigation = True
options.resilience.measure_noise_learning.num_randomizations =  32
options.resilience.measure_noise_learning.shots_per_randomization = 'auto'

# Configure ZNE
options.resilience.zne_mitigation = True
options.resilience.zne.noise_factors = (1, 3, 5)                  
options.resilience.zne.extrapolator = 'exponential'

estimator = Estimator(backend=backend, options=options)

/var/folders/_1/vyxw8wlj1wn0fhb6pl64k4lm0000gn/T/ipykernel_67110/2671142205.py:28: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=backend, options=options)
